In [1]:
import shuo
import numpy as np
import networkx as nx
import geopandas as gpd
from shuo.basics import FileNames, PickleDataWriter as pdw, UnionFind, FilesReader

File_Prefixes = ['Washington_DC', 
                 'Seattle_city_WA',
                 'Chicago_city_IL',
                 'Madison_county_AL', 
                 'Mobile_city_AL', 
                 'Napa_city_CA', 
                 'Redding_city_CA',
                 'Yuma_county_AZ']
budget = 0
current_index = 0
folder = 'data_processed'
file_prefix = File_Prefixes[current_index]
print(file_prefix)

F = FilesReader(folder, file_prefix, budget)

opt = 'minimizing_connected_components'
opt = 'minimizing_infeasible_trips'
UG, AG = F.graph(1), F.graph(2)
demand_mat = F.travel_demand_matrix()

dict_index_to_node, dict_node_to_index = F.mapping(0)
dict_index_to_zone, dict_zone_to_index = F.mapping(1)
dict_zone_to_nodes, dict_node_to_zones = F.mapping(2)

Washington_DC


In [2]:
from shuo.optimization import graph_of_minimizing_number_of_connected_components, graph_of_minimizing_number_of_infeasible_trips

eits = []
efts = []
budgets = np.arange(0, 11000, step=1000)
for budget in budgets:
    print('Current budget:', budget)
    F.update_budget(budget)
        
    # Compute the resulting graph by minimizing number of connected components
    recompute, file_name = False, F.name.graph(3, opt)
    if recompute and budget != 0:
        OG = graph_of_minimizing_number_of_connected_components(UG, AG, budget)
        OG = graph_of_minimizing_number_of_infeasible_trips(F.name.graph(1), F.name.graph(2), F.name.compressed_graph(1), F.name.compressed_graph(2), F.name.travel_demand_matrix(1), budget)
        nx.write_graphml(OG, file_name)
    else:
        OG = nx.read_graphml(file_name)
    print('There\'re', len(OG.nodes), 'nodes and', len(OG.edges), 'edges')

    # Compute and save node distance matrix
    recompute, file_name = False, F.name.node_distance_matrix(1, opt)
    if recompute and budget != 0:
        node_dist_mat = shuo.nodes_distances(OG, dict_node_to_index)
        np.save(file_name, node_dist_mat)
    else:
        node_dist_mat = np.load(file_name)
        
    # Compute and save zone distance and trips count matrix
    recompute, file_name = False, F.name.zone_trips_count_matrix(1, opt)
    if recompute and budget != 0:
        trips_count_mat = shuo.zones_distances_and_trips_count(node_dist_mat, dict_index_to_node, dict_node_to_zones, dict_zone_to_index)[1]
        np.save(file_name, trips_count_mat)
    else:
        trips_count_mat = np.load(file_name)
    
    # Show statistics
    [fea, inf] = trips_count_mat
    tot = fea + inf
    tot[tot == 0] = 1
    eit, eft = np.sum(demand_mat * (inf / tot)), np.sum(demand_mat * (fea / tot))
    eits.append(eit)
    efts.append(eft)
    print('Estimated infeasible trips:', eit)
    print('Estimated feasible trips:', eft)
    print('----------------------------------------------------------------------\n')

Current budget: 0
There're 10095 nodes and 15700 edges
Estimated infeasible trips: 187388.4444036483
Estimated feasible trips: 448673.0765963518
----------------------------------------------------------------------

Current budget: 1000
There're 10095 nodes and 15767 edges
Estimated infeasible trips: 27623.514597070363
Estimated feasible trips: 608438.0064029295
----------------------------------------------------------------------

Current budget: 2000
There're 10095 nodes and 15800 edges
Estimated infeasible trips: 20707.21377391183
Estimated feasible trips: 615354.3072260881
----------------------------------------------------------------------

Current budget: 3000
There're 10095 nodes and 15829 edges
Estimated infeasible trips: 17062.69342824195
Estimated feasible trips: 618998.8275717581
----------------------------------------------------------------------

Current budget: 4000
There're 10095 nodes and 15854 edges
Estimated infeasible trips: 14594.849544720932
Estimated feasibl

In [3]:
print('eits =', eits)
print('efts =', efts)

eits = [187388.4444036483, 27623.514597070363, 20707.21377391183, 17062.69342824195, 14594.849544720932, 13421.030010137889, 12165.924583926018, 7809.1292764316295, 7303.308908334256, 4212.989684021162, 812.7950780643882]
efts = [448673.0765963518, 608438.0064029295, 615354.3072260881, 618998.8275717581, 621466.671455279, 622640.4909898624, 623895.5964160741, 628252.3917235684, 628758.2120916657, 631848.5313159788, 635248.7259219356]
